In [ ]:
from example.discretizer.utils import IsTrafficLightNearby, IsStopSignNearby, Velocity, IsZebraNearby, LanePosition, FrontLeftObjects, FrontRightObjects, PedestrianNearby, BlockProgress
from pgeon.intention_introspector import IntentionIntrospector
from pgeon.desire import Desire
from pgeon.discretizer import Predicate
from example.discretizer.discretizer_d0 import AVDiscretizer
from example.discretizer.discretizer_d1 import AVDiscretizerD1
import pgeon.policy_graph as PG
from example.environment import SelfDrivingEnvironment
import numpy as np
import matplotlib.pyplot as plt
from typing import Set, Dict, Tuple


## Common initialization (valid for each discretiser)

In [ ]:
environment = SelfDrivingEnvironment(city='all')
discretizer_configs = {
    'a': {'obj_discretizer': 'binary', 'vel_discretizer': 'binary' },
    'b': {'obj_discretizer': 'multiple', 'vel_discretizer': 'binary'},
    'c': {'obj_discretizer': 'multiple', 'vel_discretizer': 'multiple'}
    }

default_config = {'obj_discretizer': 'multiple', 'vel_discretizer': 'multiple'}
commitment_threshold = 0.5

In [ ]:
def roc_curve(discretisers_info: Dict[str, Tuple[PG.PolicyGraph, Set[Desire]]]):
    
    plt.figure(figsize=(10, 6))

    thresholds = np.arange(0, 1, 0.1)

    for discretizer_id, (pg, desires) in discretisers_info.items():
        intention_probabilities = []
        expected_probabilities = []

        for threshold in thresholds:
            print(threshold)
            ii = IntentionIntrospector(desires, pg)

            intention_prob, expected_prob=ii.get_intention_metrics(commitment_threshold=threshold,desire=any)
            intention_probabilities.append(intention_prob)
            expected_probabilities.append(expected_prob)

        plt.plot(intention_probabilities, expected_probabilities, label=f'Discretizer D{discretizer_id}')

        
    plt.xlabel('Intention Probability for ANY desire')
    plt.ylabel('Expected Intention Probability')
    plt.title('Intention Probability vs Expected Intention Probability')
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.savefig(f'roc.png', dpi=100)

In [ ]:
def show_metrics(metrics_data, discretizer_id, type='Desire'):
    
    """
    Function to show bar plots with metrics for each desire. The metrics can be 
    desire or intention metrics, based on the value of the parameter 'type' (Desire or Intention).
    """
    desires = list(metrics_data.keys())

    fig, ax = plt.subplots(figsize=(65, 15))

    labels = []
    val1 = []
    val2 = []

    for desire in desires:
        if desire == 'any':
            val1.append(metrics_data[desire][0])
            val2.append(metrics_data[desire][1])
        else:
            val1.append(metrics_data[desire][0])
            val2.append(metrics_data[desire][1])
        labels.append(f"{desire}")

    x = np.arange(len(desires))
    width = 0.35

    rects1 = ax.bar(x - width/2, val1, width, label=f'{type} Probability')
    metric_label = 'Expected Action Prob.' if type == 'Desire' else 'Expected Intention Prob.'
    rects2 = ax.bar(x + width/2, val2, width, label=metric_label)

    ax.set_xlabel(f'Desires', fontsize=45)
    ax.set_ylabel(f'{type}s Metrics', fontsize=45)
    ax.set_title(f'Discretizer D{discretizer_id}', fontsize=50)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, fontsize=40,rotation = 30)
    
    plt.yticks(fontsize=30)

        
    ax.legend( fontsize=35, loc='upper left' )
    for rect in rects1:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2, height, f'{height:.3f}', ha='center', va='bottom', fontsize=35)

    for rect in rects2:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2, height, f'{height:.3f}', ha='center', va='bottom', fontsize=35)

    #plt.show()  
    plt.savefig(f'{type}_{discretizer_id}.png', bbox_inches = 'tight')

Desires valid for all discretisers

In [ ]:
any = Desire("any", None, set())
stop_traffic_light = Desire("stop_traffic_light", [1], {Predicate(IsTrafficLightNearby, [IsTrafficLightNearby.YES])}) #NOTE: traffic light color can be also green or yellow
stop_stop_sign = Desire("stop_stop_sign", [1], {Predicate(IsStopSignNearby, [IsStopSignNearby.YES])})
stop_zebra = Desire("stop_zebra", [1], {Predicate(IsZebraNearby, [IsZebraNearby.YES])})
lane_change = Desire("lane_change", [7,8, 2, 3 ], {Predicate(LanePosition, [LanePosition.CENTER]) }) #GAS_TURN_LEFT
turn_at_intersection = Desire("turn_at_intersection", [7,8, 2, 3 ], {Predicate(BlockProgress, [BlockProgress.INTERSECTION])})


In [ ]:
desire_test = Desire('test', [5], {IsTrafficLightNearby: [IsTrafficLightNearby.YES], Velocity: [Velocity.MEDIUM, Velocity.HIGH]})
#desire_test2 = Desire('test2', [1], {IsZebraNearby: [IsZebraNearby.NO, IsZebraNearby.YES ]})

In [ ]:
def check_state_condition(node, atom, condition_values):
    for elem in node:
        if elem.predicate == atom and elem.value[0] in condition_values:
            return True
    return False

def check_desire(node: Set[Predicate], desire_clause: Dict[str, Set[str]]) -> bool:
        # Returns None if desire is not satisfied. Else, returns probability of fulfilling desire
        #   ie: executing the action when in Node

        desire_clause_satisfied = True
        for atom, condition_values in desire_clause.items():
            desire_clause_satisfied = desire_clause_satisfied and check_state_condition(node, atom, condition_values)
            if not desire_clause_satisfied:
                return False
        return True

In [ ]:
discretizer_id = '0c'
discretizer = AVDiscretizer(
            environment,
            vel_discretization='multiple',
            obj_discretization='multiple',
            id=discretizer_id
        ) 
nodes_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_nodes.csv'
edges_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_edges.csv'
print(f'DISCRETIZER {discretizer_id} ')
pg = PG.PolicyGraph.from_nodes_and_edges(nodes_path, edges_path, environment, discretizer)
 

In [ ]:
import time
start = time.time()
for node in pg:
    print(node)
    if check_desire(node, desire_test.clause):
        print(True)
    #print(check_desire(node, desire_test2.clause))
    print()
print(f'tot time: {time.time()-start}')

## Discretisers 0a, 0b, 0c

#### Compute Desires and Intention Metrics


In [ ]:
import networkx as nx
intentions_data = {}
desires_data = {}
for discretizer_id in ['0a','0b' ,'0c', '1a','1b','1c']: #
    config = default_config
    for key in discretizer_configs:
        if key in discretizer_id:
            config = discretizer_configs[key]
            break


    DiscretizerClass = AVDiscretizer if '0' in discretizer_id else AVDiscretizerD1

    discretizer = DiscretizerClass(
            environment,
            vel_discretization=config['vel_discretizer'],
            obj_discretization=config['obj_discretizer'],
            id=discretizer_id
        ) 
    nodes_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_nodes.csv'
    edges_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_edges.csv'
    print(f'DISCRETIZER {discretizer_id} ')
    pg = PG.PolicyGraph.from_nodes_and_edges(nodes_path, edges_path, environment, discretizer)
    
    weakly_connected_components = list(nx.weakly_connected_components(pg))
    for component in weakly_connected_components:
        if len(component) == 1:
            pg.remove_node(next(iter(component))) 

    
    desires = [turn_at_intersection, stop_stop_sign, stop_traffic_light, stop_zebra, lane_change]

    if 'a' not in discretizer_id:
        desires.append(Desire("few_obstacle_avoid_l", [3, 7], {Predicate(FrontLeftObjects, [FrontLeftObjects(1, discretization=config['obj_discretizer'])]) }) )
        desires.append(Desire("few_obstacle_avoid_r", [2, 8], {Predicate(FrontRightObjects, [FrontRightObjects(1,discretization=config['obj_discretizer'])])}) )
        desires.append(Desire("many_obstacle_avoid_l", [3, 7], {Predicate(FrontLeftObjects, [FrontLeftObjects(4, discretization=config['obj_discretizer'])]) }) )
        desires.append(Desire("many_obstacle_avoid_r", [2, 8], {Predicate(FrontRightObjects, [FrontRightObjects(4,discretization=config['obj_discretizer'])])})  )
        #desires.append(Desire("overtake", [1], {Predicate(LanePosition, [LanePosition.CENTER]), Predicate(FrontRightObjects(1)) }))
    else:
        desires.append(Desire("obstacle_avoid_f", [3, 7], {Predicate(FrontLeftObjects, [FrontLeftObjects(1, discretization=config['obj_discretizer'])]) }))
        desires.append(Desire("obstacle_avoid_r", [2, 8], {Predicate(FrontRightObjects, [FrontRightObjects(1,discretization=config['obj_discretizer'])])}))

        #desires.append(Desire("overtake", [1], {Predicate(LanePosition, [LanePosition.CENTER]) }))
    ii = IntentionIntrospector(desires, pg)

    desires_data[discretizer_id] = ii.find_desires() 
    print(desires_data[discretizer_id])
    intentions_data[discretizer_id] = ii.find_intentions( commitment_threshold) #{desire.name: [self.get_intention_metrics(commitment_threshold, desire)] for desire in self.desires}
    intentions_data[discretizer_id]['any']=ii.get_intention_metrics(commitment_threshold,any)
    print(intentions_data[discretizer_id])
    print()

Questions

In [ ]:
state = pg.discretizer.str_to_state("(LanePosition(RIGHT), NextIntersection(NONE), Velocity(MOVING), Rotation(FORWARD), IsStopSignNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontRightObjects(YES), FrontLeftObjects(YES))")

In [ ]:
ii.how(pg, many_obstacle_avoid_l, state)

### Bar Plot

#### Desires

In [ ]:
for discretizer in desires_data.keys():
    show_metrics(desires_data[discretizer], discretizer, type = 'Desire')


#### Intentions

In [ ]:
for discretizer in intentions_data.keys():
    show_metrics(intentions_data[discretizer], discretizer, type = 'Intention')

## ROC Curve


Choose the discretiser with higher AIC, that is, the discretiser where the interpretability vs reliability trade-off is better

- If there is no commitment threshold in which the two metrics are decently high, it becomes apparent that the desires do not capture the agent’s behaviour. This can be either because the agent did not train correctly (making the hypothesised desires something it cannot reach) or because the agent is targeting a different set of desires. Our case is the latter. 

- The probability of manifesting intentions is low. the behaviour is not interpretable with these desires, and new ones should be considered.


After seeing seeing this, we added new desires related to OBSTACLE AVOIDANCE, which improved the AUC  for discretisers D0b and D0c.

In [ ]:
discretizers_info = {}
for discretizer_id in ['0a','0b','0c','1a','1b','1c']:
    config = default_config
    for key in discretizer_configs:
        if key in discretizer_id:
            config = discretizer_configs[key]
            break

    DiscretizerClass = AVDiscretizer if '0' in discretizer_id else AVDiscretizerD1
    discretizer = DiscretizerClass(
            environment,
            vel_discretization=config['vel_discretizer'],
            obj_discretization=config['obj_discretizer'],
            id=discretizer_id
            ) 
    nodes_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_nodes.csv'
    edges_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_edges.csv'
    pg = PG.PolicyGraph.from_nodes_and_edges(nodes_path, edges_path, environment, discretizer)
    
    
    obstacle_avoidance_l = Desire("few_left_obstacle_avoid", [3, 7], {Predicate(FrontLeftObjects, [FrontLeftObjects(1, discretization=config['obj_discretizer'])]) })
    obstacle_avoidance_r = Desire("rfew_ight_obstacle_avoid", [2, 8], {Predicate(FrontRightObjects, [FrontRightObjects(1,discretization=config['obj_discretizer'])])})
    desires = [turn_at_intersection, stop_stop_sign, stop_traffic_light, stop_zebra, lane_change, obstacle_avoidance_l, obstacle_avoidance_r]

    if 'a' not in discretizer_id:
        desires.append(Desire("many_left_obstacle_avoid", [3, 7], {Predicate(FrontLeftObjects, [FrontLeftObjects(4, discretization=config['obj_discretizer'])]) }) )
        desires.append(Desire("many_right_obstacle_avoid", [2, 8], {Predicate(FrontRightObjects, [FrontRightObjects(4,discretization=config['obj_discretizer'])])})  )

    #if '1' in discretizer_id:
        #desires.append(turn_at_intersection) 
    discretizers_info[discretizer_id] = (pg, desires)


In [ ]:
roc_curve(discretizers_info)